In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

In [3]:
torch.__version__

'1.10.0+cu111'

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


In [ ]:
PATH_OF_DATA= '/content/gdrive/"My Drive"/image_sample3'
!ls {PATH_OF_DATA}

test  train


In [ ]:
# define transformations that you wish to apply on image
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}

In [ ]:
#  Load the datasets with ImageFolder
image_datasets = {
    'train': 
    torchvision.datasets.ImageFolder(root='/content/gdrive/My Drive/image_sample3/train', transform=data_transforms['train']),
    'validation': 
    torchvision.datasets.ImageFolder(root='/content/gdrive/My Drive/image_sample3/test',  transform=data_transforms['validation'])
}

In [ ]:
print(image_datasets['train'].classes) #ok riesce a riconoscere i label giusti 
print(image_datasets['validation'].classes)

['0', '1']
['0', '1']


In [ ]:
classes = ('0', '1')

In [ ]:
#define data loaders 

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=32,
                                shuffle=True,
                                num_workers=0),  
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=32,
                                shuffle=False,
                                num_workers=0)  
}

In [ ]:
#Create the network with all layers frozen 
model = models.resnet50(pretrained=True).to(device)
    
for param in model.parameters():
    param.requires_grad = False   #freeze all layers
    
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 2)).to(device)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
#Create the network with some layers frozen (better)
model = models.resnet50(pretrained=True).to(device)

lt=8
cntr=0

for child in model.children():
  cntr +=1
if cntr < lt:
	
	for param in child.parameters():
		param.requires_grad = False
    

    
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 2)).to(device)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters())

In [ ]:
#Train the model
def train_model(model, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
    return model

In [ ]:
model_trained = train_model(model, criterion, optimizer, num_epochs=8)

Epoch 1/8
----------
train loss: 0.5214, acc: 0.7431
validation loss: 0.6908, acc: 0.4800
Epoch 2/8
----------
train loss: 0.4416, acc: 0.8624
validation loss: 0.6151, acc: 0.5800
Epoch 3/8
----------
train loss: 0.4376, acc: 0.8349
validation loss: 0.5930, acc: 0.6400
Epoch 4/8
----------
train loss: 0.4479, acc: 0.8073
validation loss: 0.5919, acc: 0.6800
Epoch 5/8
----------
train loss: 0.3436, acc: 0.8899
validation loss: 0.6666, acc: 0.5400
Epoch 6/8
----------
train loss: 0.3798, acc: 0.8349
validation loss: 0.5589, acc: 0.6800
Epoch 7/8
----------
train loss: 0.4092, acc: 0.7982
validation loss: 0.6154, acc: 0.6000
Epoch 8/8
----------
train loss: 0.4385, acc: 0.7890
validation loss: 0.8103, acc: 0.5000


In [ ]:
#Save and load the model
!mkdir models
!mkdir models/pytorch

In [ ]:
torch.save(model_trained.state_dict(), 'models/pytorch/weights.h5')

In [ ]:
model = models.resnet50(pretrained=False).to(device)
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 2)).to(device)
model.load_state_dict(torch.load('models/pytorch/weights.h5'))

<All keys matched successfully>